In [174]:
import numpy as np

In [321]:
# # Constants
# # Initial learning state
# L0 = 0.00000000000000000000012597752379894839687836118057015069025276
# # Probability of transition
# T = 0.16676161371777709074848417003522627055644989013671875000000000
# # Probability of slip
# S = 0.00044059281213343283629957358726869642850942909717559814453125
# # Probability of guess
# G = 0.00038573027973671982632231092225083557423204183578491210937500

# 	1.00000000000000000000000000000000000000000000000000000000000000
# A	1.00000000000000000000000000000000000000000000000000000000000000	0.00000000000000000000000000000000000000000000000000000000000000	0.16676161371777709074848417003522627055644989013671875000000000	0.83323838628222279822921336744911968708038330078125000000000000
# B	0.99955940718786673304663281669490970671176910400390625000000000	0.00044059281213343283629957358726869642850942909717559814453125	0.00038573027973671982632231092225083557423204183578491210937500	0.99961426972026345971755745267728343605995178222656250000000000
# 1	skillB
# PI	0.00000682511445010079077669144786977284411477739922702312469482	0.99999317488554984034720973795629106462001800537109375000000000
# A	1.00000000000000000000000000000000000000000000000000000000000000	0.00000000000000000000000000000000000000000000000000000000000000	0.00134817995705797184953500522652802828815765678882598876953125	0.99865182004294206219441321081831119954586029052734375000000000
# B	0.69999999999999995559107901499373838305473327636718750000000000	0.29999999999999998889776975374843459576368331909179687500000000	0.00000000000000000000000000000000000000000000000000000000000000	1.00000000000000000000000000000000000000000000000000000000000000


#     params = {'skills': [{'skill': '0', 
#                           'priors': np.array([0.00000000000000000000012597752379894839687836118057015069025276, 1.00000000000000000000000000000000000000000000000000000000000000]), 
#                           'transitions': np.array([1.        , 0.        , 0.16676161371777709074848417003522627055644989013671875000000000, 0.83323838628222279822921336744911968708038330078125000000000000]), 
#                           'emissions': np.array([0.99955940718786673304663281669490970671176910400390625000000000, 0.00044059281213343283629957358726869642850942909717559814453125, 0.00038573027973671982632231092225083557423204183578491210937500, 0.99961426972026345971755745267728343605995178222656250000000000])}, 
#                          {'skill': '2', 
#                           'priors': np.array([0.00000682511445010079077669144786977284411477739922702312469482, 0.99999317488554984034720973795629106462001800537109375000000000]), 
#                           'transitions': np.array([1.        , 0.        , 0.00134817995705797184953500522652802828815765678882598876953125, 0.99865182004294206219441321081831119954586029052734375000000000]), 
#                           'emissions': np.array([0.69999999999999995559107901499373838305473327636718750000000000, 0.29999999999999998889776975374843459576368331909179687500000000, 0.00000000000000000000000000000000000000000000000000000000000000, 1.00000000000000000000000000000000000000000000000000000000000000])}], 
#               'SolverId': '1.1', 'nK': '2', 'nG': '50', 'nS': '2', 'nO': '2', 'nZ': '1', 'Null skill ratios': '  1.0000000\t  0.0000000'}

    
params = {'skills': [{'skill': '0', 
                      'priors': np.array([0., 1.]), 
                      'transitions': np.array([1.        , 0.        , 0.16676161, 0.83323839]), 
                      'emissions': np.array([9.99559407e-01, 4.40592800e-04, 3.85730300e-04, 9.99614270e-01])}, 
                     {'skill': '1', 
                      'priors': np.array([6.82510000e-06, 9.99993175e-01]), 
                      'transitions': np.array([1.        , 0.        , 0.00134818, 0.99865182]), 
                      'emissions': np.array([0.7, 0.3, 0. , 1. ])}]}

In [278]:
def correct(learning_state, skills_idx):
    learning_correct = learning_state * (1 - S[skills_idx])
    guess_correct = ((1 - learning_state) * G[skills_idx])
    learning_evidence = learning_correct/(learning_correct + guess_correct)
    return learning_evidence

def incorrect(learning_state, skills_idx):
    learning_incorrect = learning_state * S[skills_idx]
    guess_incorrect = ((1 - learning_state) * (1 - G[skills_idx]))
    learning_evidence = learning_incorrect/(learning_incorrect + guess_incorrect)
    return learning_evidence
    
def update(learning_state, skills_idx, iscorrect=True):
    if iscorrect:
        learning_evidence = correct(learning_state, skills_idx)
    else:
        learning_evidence = incorrect(learning_state, skills_idx)
#     print("Learning evidence: %s" % (learning_evidence))
    learning_state = learning_evidence + ((1 - learning_evidence) * T[skills_idx])
    return learning_state

def get_correct_prob(learning_state, skills_idx):
    return (1-S[skills_idx])*learning_state + G[skills_idx]*(1-learning_state)

In [323]:
# Contruct params matrix
skills = np.zeros(len(params['skills']))
learning_state = np.zeros(len(params['skills']))
T = np.zeros(len(params['skills']))
S = np.zeros(len(params['skills']))
G = np.zeros(len(params['skills']))
for idx, skill in enumerate(params['skills']):
    skills[idx] = skill['skill']
    L0[idx] = skill['priors'][0]
    T[idx] = skill['transitions'][2]
    S[idx] = skill['emissions'][1]
    G[idx] = skill['emissions'][2]

outcomes = [2,2,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,2]
    
loglikelihood = np.zeros(len(outcomes))
ll_local = 0
outcome_prob_skill = np.zeros((L0.shape[0], len(outcomes)))
outcome_prob = np.zeros((len(outcomes),2))

q_matrix = np.array([[1,0]]*len(outcomes))
q_matrix[[0,1],1] = 1

for idx, outcome in enumerate(outcomes):
    skills_idx = np.where(q_matrix[idx,:] == 1)
    
    # Sliced learning states (skill is present in this question)
    l_sliced = learning_state[skills_idx]
    
    # Calculate chance of being correct for each skill
    outcome_prob_skill[skills_idx, idx] = get_correct_prob(l_sliced, skills_idx)
    outcome_prob[idx,0] = outcome_prob_skill[:,idx].sum()/skills_idx[0].shape[0]
    outcome_prob[idx,1] = 1-outcome_prob[idx,0]
    
    # Calculate LL and update state
    if outcome == 1:
        ll_local = np.log(outcome_prob[idx,0])
        l_sliced = update(l_sliced, skills_idx, True)
    else:
        ll_local = np.log(outcome_prob[idx,1])
        l_sliced = update(l_sliced, skills_idx, False)
    learning_state[skills_idx] = l_sliced
        
    loglikelihood[idx] += ll_local
    print("%d\t%.5f\t%.5f\t%.5f\t%.5f\t" % (outcome, outcome_prob[idx,0], outcome_prob[idx,1], ll_local, loglikelihood.sum()))

2	0.00019	0.99981	-0.00019	-0.00019	
2	0.08398	0.91602	-0.08771	-0.08791	
2	0.16708	0.83292	-0.18282	-0.27073	
2	0.16708	0.83292	-0.18282	-0.45355	
2	0.16708	0.83292	-0.18282	-0.63637	
2	0.16708	0.83292	-0.18282	-0.81919	
1	0.16708	0.83292	-1.78926	-2.60846	
1	0.99796	0.00204	-0.00204	-2.61050	
1	0.99956	0.00044	-0.00044	-2.61094	
1	0.99956	0.00044	-0.00044	-2.61138	
1	0.99956	0.00044	-0.00044	-2.61182	
1	0.99956	0.00044	-0.00044	-2.61226	
1	0.99956	0.00044	-0.00044	-2.61270	
1	0.99956	0.00044	-0.00044	-2.61314	
1	0.99956	0.00044	-0.00044	-2.61359	
1	0.99956	0.00044	-0.00044	-2.61403	
1	0.99956	0.00044	-0.00044	-2.61447	
1	0.99956	0.00044	-0.00044	-2.61491	
1	0.99956	0.00044	-0.00044	-2.61535	
2	0.99956	0.00044	-7.72739	-10.34274	


In [326]:
aic = -2*loglikelihood.sum() + 2*4*skills.shape[0]
aic

36.685475118198525

In [327]:
bic = -2*loglikelihood.sum() + 4*skills.shape[0]*np.log(len(outcomes))
bic

44.651333306630455

In [328]:
estimated_outcome = np.argmax(outcome_prob, axis=1)
acc = (estimated_outcome == np.asarray(outcomes)-1).sum()/len(outcomes)
acc

0.9

In [329]:
rmse = ((1-outcome_prob[range(len(outcomes)), (np.asarray(outcomes)-1)])**2).sum()
rmse = np.sqrt(rmse/len(outcomes))
rmse

0.3009647032927156